# Exploration of Missed Tackles

## Why Missed Tackles

Tackling is a critical part of every football play. The one goal of the defense is to reach the ball carrier and bring them to the ground. This goal has two parts. First the defender must arrive in the same location as the ball carrier, but once they are there it is still imperitive to complete the tackle. Sometimes the defender is unable to bring the ball carrier to the ground and this is considered a missed tackle. This exploration considers the parts of the tackle and looks for indicators of missed tackles prior to contact.

## Metrics

### Parts of a Tackle

### Calculation of Metrics

## Distinguishing Missed Tackles